# Introduction

Ce TP continue le TP précédent. Nous allons reprendre d'ailleurs les mêmes données et commencer la mise en oeuvre d'un modèle de Markov pour la prédiction des étiquettes: 
* une observation est une phrase, représentée comme une séquence de variables aléatoires, une par mot de la phrase
* à cette observation est associée une séquence de variables aléatoires représentant les étiquettes, une par mot de la phrase également

On suppose que la séquence d'observation (une phrase) est générée par un modèle de Markov caché. Les variables cachées sont donc les étiquettes à inférer. Nous allons commencer par écrire une classe python pour représenter le HMM. Cette classe évoluera au fil des TPs. 

Pour cela le code de départ suivant est donné. Afin d'initialiser un HMM, nous devons connaitre : 
- l'ensemble des états (ou *state_list*), dans notre cas l'ensemble des étiquettes grammaticales;
- l'ensemble des observations (ou *observation_list*), dans notre cas l'ensemble des mots connus; tous les autres mots seront remplacés par l'élément spécial *UNK* qui fait partie de l'ensemble des observations. 

Enfin, en interne il est plus facile d'indexer les mots et et les états par des entiers. Ainsi à chaque éléments de respectivement l'ensemble des états et l'ensemble des observations, est associé un indice. Cela nous permet de tout traiter en "matricielle". 

In [1]:
import nltk
from numpy import array, ones, zeros
import sys
from __future__ import division

# Some words in test could be unseen during training, or out of the vocabulary (OOV) even during the training. 
# To manage OOVs, all words out the vocabulary are mapped on a special token: UNK defined as follows: 
UNK = "<unk>" 
UNKid = 0 

class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None):
            """Builds a new Hidden Markov Model
            state_list is the list of state symbols [q_0...q_(N-1)]
            observation_list is the list of observation symbols [v_0...v_(M-1)]
            transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print "HMM creating with: "
            self.N = len(state_list) # The number of states
            self.M = len(observation_list) # The number of words in the vocabulary
            print str(self.N)+" states"
            print str(self.M)+" observations"
            self.omega_Y = state_list # Keep the vocabulary of tags
            self.omega_X = observation_list # Keep the vocabulary of tags
            # Init. of the 3 distributions : observation, transition and initial states
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            # Since everything will be stored in numpy arrays, it is more convenient and compact to 
            # handle words and tags as indices (integer) for a direct access. However, we also need 
            # to keep the mapping between strings (word or tag) and indices. 
            self.make_indexes()
            

        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities arrays"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
      


# Interface avec les données et apprentissage supervisé

Ainsi pour initialiser un HMM, nous allons devoir lire les données (chose faîte lors du TP précédent): 
* écrire une fonction permettant d'initialiser le HMM à partir des données d'apprentissage
* écrire une fonction *apprentissage_supervisé* qui permet d'estimer les paramètres 

Dans un premier temps, nous limiterons, comme lors du TP précédent, le vocabulaire aux mots apparaissant 10 fois ou plus. Les autres mots sont tous remplacés par la même forme *unk*

Pour cela, le plan de travail peut être envisagé ainsi: 
* Lire les données puis générer un corpus de **train** (80%) puis de **test** (10%)
* écrire une fonction qui créer à partir des données d'apprentissage (**train**), tous les comptes nécessaires pour l'estimation supervisée des paramètres du HMM
* écrire 3 fonctions qui estimes les paramètres à partir des comptes, une fonction par distribution: observation, transition, état initial. 
* écrire une fonction qui reprend le tout et qui estime tous les paramètres du HMM

In [2]:
import nltk
import pickle
mydata = pickle.load( open( "brown.save.p", "rb" ) )
data=mydata
# Lire les données
#data = nltk.corpus.brown.tagged_sents(tagset='universal')


In [3]:
data=mydata

Générer les corpus train (80%), valid (10%) et test (10%)

In [4]:
import random

train_set = []
valid_set = []
test_set = []
for sentence in data:
    r = random.random()
    if r < 0.8:
        train_set.append(sentence)
    elif r < 0.8 + 0.1:
        valid_set.append(sentence)
    else:
        test_set.append(sentence)

In [5]:
train_set[0]

[(u'The', u'DET'),
 (u'Fulton', u'NOUN'),
 (u'County', u'NOUN'),
 (u'Grand', u'ADJ'),
 ('<unk>', u'NOUN'),
 (u'said', u'VERB'),
 (u'Friday', u'NOUN'),
 (u'an', u'DET'),
 (u'investigation', u'NOUN'),
 (u'of', u'ADP'),
 ('<unk>', u'NOUN'),
 (u'recent', u'ADJ'),
 (u'primary', u'NOUN'),
 (u'election', u'NOUN'),
 (u'produced', u'VERB'),
 (u'``', u'.'),
 (u'no', u'DET'),
 (u'evidence', u'NOUN'),
 (u"''", u'.'),
 (u'that', u'ADP'),
 (u'any', u'DET'),
 ('<unk>', u'NOUN'),
 (u'took', u'VERB'),
 (u'place', u'NOUN'),
 (u'.', u'.')]

Récupérer la liste deses mots et des étiquettes

In [6]:
import IPython
import copy
data2=[]
# data2.append(copy.copy(data[0][0:15]))
# data2.append(copy.copy(data[0][0:15]))
# print data2
words = set() 
tags = set()
for sentence in data:
    for elt in sentence:
        words.add(elt[0]) 
        tags.add(elt[1])

words = list(words)
tags = list(tags)

words_index=dict();
tags_index=dict();
words_list=list();
tags_list=list();
   
j=0;
for word in words:
    if word not in words_index:
        words_index[word]=j
        words_list.append(word)
        j=j+1
j=0;
for tag in tags:
    if tag not in tags_index:
        tags_index[tag]=j
        tags_list.append(tag)
        j=j+1

print tags_index
print words_index
# S=list(len(tags))
# for sentence in train_set:
#     S[tags_index(sentence[0][0])]+=1;

        


{u'ADV': 0, u'NOUN': 1, u'NUM': 9, u'ADP': 2, u'PRT': 6, u'DET': 4, u'.': 5, u'PRON': 3, u'VERB': 7, u'X': 8, u'CONJ': 10, u'ADJ': 11}
{u'writings': 0, u'fig.': 1, u'yellow': 2, u'Heights': 3, u'four': 4, u'woods': 5, u'hanging': 6, u'Until': 7, u'aggression': 8, u'marching': 9, u'Foundation': 52, u'eligible': 11, u'electricity': 12, u'Myra': 2771, u'Casey': 14, u'Belgians': 15, u'Western': 16, u'hormone': 17, u'co-operation': 18, u'regional': 19, u'uncertain': 20, u'screaming': 152, u'disturb': 22, u'scholar': 163, u'Less': 25, u'wooden': 26, u'persisted': 27, u'Does': 28, u'succession': 29, u'Paul': 30, u'commented': 31, u'non-violent': 32, u'charter': 33, u'bill': 5957, u'tired': 35, u'Some': 6387, u'empirical': 36, u'tires': 37, u'elegant': 38, u'second': 39, u'sailed': 40, u'errors': 41, u'Lucille': 42, u'cooking': 43, u'contributed': 44, u'fingers': 45, u'Hamilton': 46, u'College': 5960, u'contributes': 299, u'Presidential': 49, u'specialist': 50, u'hero': 51, u'reporter': 318, u

In [7]:
tags_index
# data[0]


{u'.': 5,
 u'ADJ': 11,
 u'ADP': 2,
 u'ADV': 0,
 u'CONJ': 10,
 u'DET': 4,
 u'NOUN': 1,
 u'NUM': 9,
 u'PRON': 3,
 u'PRT': 6,
 u'VERB': 7,
 u'X': 8}

# Estimation des paramètres
Les paramètres sont : pi_i, a_ij et b_ij (tels que définis plus haut et dans le cours).
On va commencer par pi_i (car c'est en 1D donc apparement plus facile).

Pour i=0..N-1, pi_i = P(Y0 = state_i) = nombre de 1ers mots qui ont une etiquette state_i / nombre de phrases. nombre de phrases = len(train_set), donc il faut juste compter le nombre de 1ers mots qui ont une etiquette state_i, pour chaque étiquette i (i=1..N)

In [8]:
import types
import pdb
def compter(self,train_set,tags_index,words_index):
#     tags_index=self.omega_Y
#     words_index=self.omega_X
    initial_tag_count =zeros(len(tags_index))  # tag: # of occurrences
    #compute the list of tags
    transition_matrix=zeros((len(tags_index),len(tags_index)))
    observation_matrix=zeros((len(words_index),len(tags_index)))
    
    for sentence in train_set:
            init_word, init_tag = sentence[0]
            if tags_index[init_tag] not in initial_tag_count:
                initial_tag_count[tags_index[init_tag]]=1 
            else:
                initial_tag_count[tags_index[init_tag]]+=1

            for i in range(len(sentence)-1):
                transition_matrix[tags_index[sentence[i][1]],tags_index[sentence[i+1][1]]]+=1
    #             print transition_matrix[tags_index[sentence[i][1]],tags_index[sentence[i+1][1]]]
            for i in range(len(sentence)):
                observation_matrix[words_index[sentence[i][0]],tags_index[sentence[i][1]]]+=1
    transition_matrix=transition_matrix;
    initial_tag_count=initial_tag_count;
    observation_matrix=observation_matrix;
#     pdb.set_trace()    
    initial_tag_count=initial_tag_count/len(train_set)
    
    for i in range(len(tags)):
        transition_matrix[i,:]=transition_matrix[i,:]/(transition_matrix.sum(axis=1))[i]
    for i in range(len(words)):
        observation_matrix[i,:]=observation_matrix[i,:]/(observation_matrix.sum(axis=1))[i]   
    self.observation_proba=observation_matrix
    self.transition_proba=transition_matrix
    self.initial_state_proba=initial_tag_count
    
    print('finished counting')
    return initial_tag_count,observation_matrix,transition_matrix
hmm=HMM(tags_list,words_list)
hmm.compter=types.MethodType(compter,hmm,HMM)
hmm.compter(data,tags_index,words_index)
# initial_tag_count,observation_matrix,transition_matrix=hmm.compter(train_set,tags_index,words_index)

HMM creating with: 
12 states
8896 observations
finished counting


(array([  1.74398326e-05,   1.41140565e-01,   1.74398326e-05,
          1.74398326e-05,   1.74398326e-05,   1.74398326e-05,
          1.74398326e-05,   1.74398326e-05,   1.74398326e-05,
          1.74398326e-05,   1.74398326e-05,   1.74398326e-05]),
 array([[ 0.        ,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.07692308,  0.        , ...,  0.        ,
          0.        ,  0.90384615],
        ..., 
        [ 0.        ,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  1.        ],
        [ 0.36842105,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.57894737]]),
 array([[  9.68711646e-02,   3.28358740e-02,   1.41927106e-01,
           4.83644320e-02,   7.35694338e-02,   1.70227147e-

In [9]:
# print tags_index
# print observation_matrix

range(len(tags))
len(tags)

12

# Exercice : Algorithme de Viterbi

La question qui se pose est comment calculer la meilleure séquence d'étiquettes pour une phrase donnée connaissant les paramètres du HMM. Par meilleure, on entend la séquence d'étiquettes (ou d'états) la plus probable connaissant la séquence d'obervation. 

Proposer et implémenter un algorithme répondant à cette question. Pour vous aider à démarrer, cet algorithme s'appelle Viterbi et regardez cette vidéo https://www.youtube.com/watch?v=RwwfUICZLsA, pour comprendre comment il opère. 

# TODO pour le 10/10/2017

* Finir la partie interface (qui comprend l'apprentissage supervisé)
* Regarder la vidéo et implémenter l'algorithme de Viterbi

In [10]:

import numpy
epsilon=0.0000001
def viterbiAlgo(self,sentence):
    self.omega_X
    logdelta=zeros((self.N,len(sentence)))
    psi=zeros((self.N,len(sentence)))
    #Initialisation
    for i in range(self.N):
        logdelta[i,0]=numpy.log(self.observation_proba[self.X_index[sentence[0][0]],i]+epsilon)+numpy.log(self.initial_state_proba[i]+epsilon)
        psi[i,0]=0    
#     pdb.set_trace()
    for k in range(1,len(sentence)):
        for i in range(self.N):
            value=logdelta[0:self.N-1,k-1]+numpy.log(self.observation_proba[self.X_index[sentence[k][0]],i]+epsilon)+numpy.log(epsilon+self.transition_proba[i,0:self.N-1])
            psi[i,k]=int(numpy.argmax(value))
            logdelta[i,k]=max(value)
    sequence=[]
    MaxIndex=int(numpy.argmax(logdelta[i,:]))
#     pdb.set_trace()
    totalDelta=max(logdelta[i,:])
    sequence=[MaxIndex]
    for k in range(len(sentence)-1,0,-1):
        sequence=[int(psi[MaxIndex,k])]+sequence
        MaxIndex=int(psi[int(MaxIndex),k])
#     pdb.set_trace()
    labelsSequence=[]
    for k in range(0,len(sentence)):
        labelsSequence.append(self.omega_Y[sequence[k]])
        
    return psi,logdelta,labelsSequence
hmm.viterbiAlgo=viterbiAlgo
psi,delta,labelsSequence=hmm.viterbiAlgo(hmm,test_set[2])

# hmm.viterbiAlgo=types.MethodType(viterbiAlgo,hmm,HMM)

In [13]:
len(labelsSequence)
sentenceLabels=[x[1] for x in test_set[2] ]
mat=[(labelsSequence[i],sentenceLabels[i]) for i in range(len(labelsSequence))]
mat

# labelsSequence

# test_set[2]
# Mask= [labelsSequence[i]==sentenceLabels[i] for i in range(len(labelsSequence))]  
# Mask

[(u'DET', u'NOUN'),
 (u'ADP', u'NOUN'),
 (u'.', u'NOUN'),
 (u'DET', u'NOUN'),
 (u'VERB', u'NOUN'),
 (u'NOUN', u'NOUN'),
 (u'ADP', u'VERB'),
 (u'DET', u'ADP'),
 (u'VERB', u'NOUN'),
 (u'.', u'VERB'),
 (u'ADV', u'VERB'),
 (u'.', u'PRT'),
 (u'VERB', u'ADP'),
 (u'DET', u'DET'),
 (u'VERB', u'NOUN'),
 (u'NOUN', u'NOUN'),
 (u'DET', u'PRT'),
 (u'VERB', u'VERB'),
 (u'VERB', u'VERB'),
 (u'PRON', u'NOUN'),
 (u'ADP', u'NUM'),
 (u'NOUN', u'ADP')]

In [12]:
error=[];
for sentence in test_set:
    psi,delta,labelsSequence=hmm.viterbiAlgo(hmm,sentence)
    sentenceLabels=[x[1] for x in sentence ]
#     mat=[(labelsSequence[i],sentenceLabels[i]) for i in range(len(labelsSequence))]
    error.append( 1-(sum([labelsSequence[i]==sentenceLabels[i] for i in range(len(labelsSequence))]))/len(sentenceLabels))
# Error=error/len(test_set)
sum(error)/len(error)

0.23733955394811895